# Extended Biosignal Preprocessing Pipeline Tutorial

Welcome to this extended tutorial on building a raw data preprocessing pipeline using the ECG_ACORAI codebase. In this notebook, you'll learn how to simulate, preprocess, denoise, and visualize different biosignals—including:

- **Electrodermal Activity (EDA)**
- **Electromyogram (EMG)**
- **Respiration**
- **Electrocardiogram (ECG)**

We cover both basic filtering techniques and more advanced denoising methods (e.g. wavelet thresholding for ECG), discuss potential error scenarios, and suggest next steps to integrate these preprocessed signals into a machine learning pipeline.

Throughout the notebook, you'll see detailed explanations for each step, code examples, and plots for visualization. This tutorial is designed both as a guide and a reference for extending the processing pipeline further.

## Table of Contents

1. [Environment Setup and Imports](#environment-setup-and-imports)
2. [Simulating Raw Biosignals](#simulating-raw-biosignals)
3. [Preprocessing Functions and Theory](#preprocessing-functions-and-theory)
    - [Low-pass Filtering for EDA and Respiration](#low-pass-filtering-for-eda-and-respiration)
    - [Bandpass Filtering and Processing for EMG](#bandpass-filtering-and-processing-for-emg)
    - [Wavelet Denoising for ECG](#wavelet-denoising-for-ecg)
4. [Detailed Scenarios and Visualizations](#detailed-scenarios-and-visualizations)
    - [Scenario 1: EDA & Respiration Filtering](#scenario-1-eda--respiration-filtering)
    - [Scenario 2: EMG Denoising](#scenario-2-emg-denoising)
    - [Scenario 3: Advanced ECG Denoising](#scenario-3-advanced-ecg-denoising)
5. [Integration into an ML Pipeline](#integration-into-an-ml-pipeline)
6. [Troubleshooting and Optimization Tips](#troubleshooting-and-optimization-tips)
7. [Conclusion and Next Steps](#conclusion-and-next-steps)

## 1. Environment Setup and Imports

Before we start, ensure that you have installed the repository requirements (e.g., via `uv pip install -r requirements.txt`) and have cloned the ECG_ACORAI repository. In this section, we import key libraries:

- **NumPy & SciPy** for numerical processing and filtering
- **Matplotlib** for plotting and visualization
- **NeuroKit2** for simulating biosignals
- **PyTorch** for tensor-based operations
- **ECG_ACORAI modules** (e.g. `wavelet_denoise` from `ecg_processor_torch.advanced_denoising`)

Let's import these now:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import neurokit2 as nk
import torch

# Import the advanced ECG denoising module from the repository
from ecg_processor_torch.advanced_denoising import wavelet_denoise
from ecg_processor_torch.config import ECGConfig

# Configure matplotlib for inline plotting
%matplotlib inline

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("Setup complete. Using ECG sampling rate:", ECGConfig.DEFAULT_SAMPLING_RATE)

## 2. Simulating Raw Biosignals

In real-world scenarios, biosignals come from sensors stored in files or streamed in real time. For demonstration, we'll simulate:

- **EDA**: A slow-varying signal with high-frequency noise superimposed
- **EMG**: A high-frequency signal with muscular bursts (sine wave plus noise)
- **Respiration**: Low-frequency oscillation with perturbations
- **ECG**: Simulated using NeuroKit2 (noisy version)

Each signal is sampled at 500 Hz over a 10-second duration. Let's simulate these signals and plot them.

In [ ]:
# Simulation parameters
fs = 500  # Sampling rate (Hz)
duration = 10  # seconds
t = np.linspace(0, duration, fs * duration)

# Simulate EDA: Slow sinusoid with extra noise
eda = 0.5 * np.sin(0.2 * 2 * np.pi * t) + 0.05 * np.random.randn(len(t))

# Simulate EMG: High-frequency sinusoid with noise
emg = 0.1 * np.sin(20 * 2 * np.pi * t) + 0.02 * np.random.randn(len(t))

# Simulate Respiration: Low-frequency oscillation
resp = 0.3 * np.sin(0.5 * 2 * np.pi * t) + 0.03 * np.random.randn(len(t))

# Simulate ECG using NeuroKit2 (noisy version)
ecg = nk.ecg_simulate(duration=10, sampling_rate=fs, noise=0.1)

# Plot all simulated signals
plt.figure(figsize=(14, 12))

plt.subplot(4, 1, 1)
plt.plot(t, eda, color='tab:blue')
plt.title('Simulated EDA Signal (Raw)')

plt.subplot(4, 1, 2)
plt.plot(t, emg, color='tab:orange')
plt.title('Simulated EMG Signal (Raw)')

plt.subplot(4, 1, 3)
plt.plot(t, resp, color='tab:green')
plt.title('Simulated Respiration Signal (Raw)')

plt.subplot(4, 1, 4)
plt.plot(t, ecg, color='tab:red')
plt.title('Simulated ECG Signal (Raw)')

plt.tight_layout()
plt.show()

## 3. Preprocessing Functions and Theory

Raw biosignals often have noise and artifacts. We'll define several preprocessing functions:

### Low-pass Filtering (EDA and Respiration)

A low-pass filter (Butterworth) removes high-frequency noise from slow-varying signals.

### Bandpass Filtering and Processing (EMG)

EMG signals are processed using a bandpass filter to remove noise, followed by rectification and smoothing.

### Advanced Wavelet Denoising (ECG)

Wavelet denoising decomposes the signal, thresholds noisy coefficients, and reconstructs a cleaner signal. Our repository's `wavelet_denoise` function demonstrates this method.

In [ ]:
def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def filter_signal(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Preprocessing for EDA and Respiration
def clean_eda(signal, fs):
    cutoff = 0.5  # Hz
    return filter_signal(signal, cutoff, fs, order=3)

def clean_respiration(signal, fs):
    cutoff = 1  # Hz
    return filter_signal(signal, cutoff, fs, order=3)

# EMG Processing: Bandpass filter, rectification, and smoothing
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def denoise_emg(signal, fs):
    lowcut = 20   # Hz
    highcut = 150 # Hz (ensure highcut < Nyquist)
    b, a = butter_bandpass(lowcut, highcut, fs, order=4)
    filtered = filtfilt(b, a, signal)
    rectified = np.abs(filtered)
    smooth = filter_signal(rectified, cutoff=10, fs=fs, order=3)
    return smooth

## 4. Detailed Scenarios and Visualizations

Below, we explore scenarios for each signal, comparing raw and preprocessed signals.

### Scenario 1: EDA & Respiration Filtering

We apply low-pass filtering for EDA and Respiration. Summary statistics (mean, std) are computed before and after filtering.

In [ ]:
# Process EDA and Respiration
eda_clean = clean_eda(eda, fs)
resp_clean = clean_respiration(resp, fs)

print("EDA - Raw: mean={:.3f}, std={:.3f}".format(np.mean(eda), np.std(eda)))
print("EDA - Clean: mean={:.3f}, std={:.3f}".format(np.mean(eda_clean), np.std(eda_clean)))

print("Respiration - Raw: mean={:.3f}, std={:.3f}".format(np.mean(resp), np.std(resp)))
print("Respiration - Clean: mean={:.3f}, std={:.3f}".format(np.mean(resp_clean), np.std(resp_clean)))

plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(t, eda, label='Raw EDA', alpha=0.6)
plt.plot(t, eda_clean, label='Cleaned EDA', linewidth=2)
plt.title('EDA Signal Processing')
plt.xlabel('Time (s)')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(t, resp, label='Raw Respiration', alpha=0.6)
plt.plot(t, resp_clean, label='Cleaned Respiration', linewidth=2)
plt.title('Respiration Signal Processing')
plt.xlabel('Time (s)')
plt.legend()

plt.tight_layout()
plt.show()

### Scenario 2: EMG Denoising

We apply bandpass filtering, rectification, and smoothing to denoise the EMG signal. The resulting signal highlights muscle activity bursts.

In [ ]:
# Process EMG signal
emg_clean = denoise_emg(emg, fs)

plt.figure(figsize=(14, 4))
plt.plot(t, emg, label='Raw EMG', alpha=0.6)
plt.plot(t, emg_clean, label='Denoised EMG', linewidth=2)
plt.title('EMG Signal Processing')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

### Scenario 3: Advanced ECG Denoising with Wavelet Thresholding

The advanced wavelet denoising method decomposes the ECG signal using wavelets, thresholds noisy components, and reconstructs a cleaner version. We demonstrate converting between NumPy and Torch tensors and plotting the result.

In [ ]:
# Simulate a noisy ECG signal
ecg_noisy = nk.ecg_simulate(duration=10, sampling_rate=fs, noise=0.1)

# Convert ECG signal to Torch tensor
ecg_noisy_tensor = torch.tensor(ecg_noisy, dtype=torch.float32)

# Apply advanced wavelet denoising
ecg_denoised_tensor = wavelet_denoise(ecg_noisy_tensor)

# Convert denoised signal back to NumPy for plotting
ecg_denoised = ecg_denoised_tensor.cpu().numpy()

plt.figure(figsize=(14, 4))
plt.plot(t, ecg_noisy, label='Noisy ECG', alpha=0.6)
plt.plot(t, ecg_denoised, label='Denoised ECG', linewidth=2)
plt.title('Advanced ECG Denoising with Wavelet Thresholding')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

## 5. Integration into an ML Pipeline

With preprocessed signals, you can extract features and build a machine learning pipeline. For example:

- **Feature Extraction:** Compute statistical or spectral features from cleaned signals.
- **Model Training:** Use those features to train classifiers (e.g. a transformer-based ECG classifier).

Below is a schematic pseudo-code snippet to illustrate integration:

n# Convert raw ECG to a torch tensor (if not already)
ecg_tensor = torch.tensor(ecg_raw, dtype=torch.float32)

# Preprocess using an ECGPreprocessor
from ecg_processor_torch.ecg_preprocessor import ECGPreprocessor
preprocessor = ECGPreprocessor()
ecg_preprocessed = preprocessor.process_signal(ecg_tensor)

# Extract features (user-defined routine)
features = extract_features(ecg_preprocessed)

# Predict using a trained classifier
from ecg_processor_torch.ecg_transformer_classifier_torch import ECGTransformerClassifier
classifier = ECGTransformerClassifier(...)
predictions = classifier.predict(features)

# Convert raw ECG to a torch tensor (if not already)
ecg_tensor = torch.tensor(ecg_raw, dtype=torch.float32)

# Preprocess using an ECGPreprocessor
from ecg_processor_torch.ecg_preprocessor import ECGPreprocessor
preprocessor = ECGPreprocessor()
ecg_preprocessed = preprocessor.process_signal(ecg_tensor)

# Extract features (user-defined routine)
features = extract_features(ecg_preprocessed)

# Predict using a trained classifier
from ecg_processor_torch.ecg_transformer_classifier_torch import ECGTransformerClassifier
classifier = ECGTransformerClassifier(...)
predictions = classifier.predict(features)
```

## 6. Troubleshooting and Optimization Tips

Here are some tips to ensure your pipeline runs smoothly:

- Sampling Frequency & Filter Parameters: Ensure cutoff frequencies are properly normalized (cutoff < Nyquist).
- Tensor Conversions: Use np.ascontiguousarray() if encountering negative strides during conversion between NumPy and Torch.
- Modular Design: Separate each step (filtering, denoising, feature extraction) for easier testing.
- Real-time Processing: Optimize functions for batch processing if deploying in real time.
- Error Handling: Implement logging or exception handling to catch shape mismatches or device-related errors.

## 7. Conclusion and Next Steps

In this notebook, we demonstrated:

- Simulation of multiple biosignals (EDA, EMG, Respiration, ECG)
- Application of basic filtering and advanced denoising techniques
- Visualization and summary statistics comparing raw and cleaned signals
- Integration insights for an ML pipeline

Next steps:

- Expand feature extraction and classification routines
- Incorporate artifact removal techniques
- Optimize the pipeline for real-time deployment

Thank you for following along. Happy coding!